- Author: Linna An
- Environment: `common_use`
- Some prediction xml adapted from protocol generated by Dmitri & Derrick & Bcov
- Version: 22.06.21
- For ligands docking to pseudocycles to select suitable docks
- all enumerate loops are for generate cmds for test. (aka, use n < 1, generate 1 cmd for test)
- optimize 22.06.07: 
    - change all AF2 to batch run. 
    - change pyrosetta to Biopython to get sequence to save time. Get seq from pose use ~ 0.6sec, Biopython use 0.1 sec
- optimize 22.06.21: 
    - Optimized predictor1, speed saved from 1.7sec to 1.28sec (jojo). 
    - Optimized predictor2, speed saved from 32.5sec to 4.3sec (jojo).

In [2]:
import os
import sys
import re
import random
import numpy as np
import pandas as pd
import math
import glob
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
from shutil import copyfile
import tempfile
import shutil
from collections import defaultdict
import xarray
import gzip
import time
import json
from Bio import SeqIO

from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from scipy.cluster.hierarchy import dendrogram
import scipy as sp
import sklearn as sk

from sklearn.cluster import KMeans
import sklearn.metrics as sm
%matplotlib inline
pd.options.display.max_colwidth = 1000

sys.path.append('./software/')
from get_rmsds_functions import TMalign
from get_rmsds_functions import get_RMSD
import check_interactions_to_lig

sys.path.append('./lib/')
from libSlurm import make_submit_file
from libSlurm import make_dist_plots
from libSlurm import sed_inplace
from libSlurm import grep
from libSlurm import get_flags
from libSlurm import plot_ROC
from libCommonJupyterFunc import get_total_scores
from check_interactions_to_lig import get_lig_num
from check_interactions_to_lig import get_hb2lig
import silent_tools # Bcov: https://github.com/bcov77/silent_tools.git

In [7]:
home_dir = '/home/linnaan/software/Pseudocycle_small_molecule_binder/' # change to where your scripts and ligands are, if you pull from git, then it should just be the git directory
scratch_dir = '/net/scratch/linnaan/New_AF2_scaffolds/220214_dock/' # change to where you want to put your intermediate outputs

#ligands
# format: {name,[genot_param,ref2015_param, [rotamers],requirements,required_atms]}
# required_atms should be adjusted based on rifgen rif size results. too small rif
# hurt rifdock significantly
ligands = {
            'T44':['/home/linnaan/ligands/T44/sel_low_energy/good_to_use/T44_0001_relax_charged_genpot.params',\
                  '/home/linnaan/ligands/T44/sel_low_energy/good_to_use/T44_ref2015.params',\
                  [i for i in glob.glob('/home/linnaan/ligands/T44/sel_low_energy/good_to_use/T44_*_relax_charged_ref2015.pdb') if 'linker' not in i],\
                  '2,3,5,6,8,9','N1,H8,H9,H11,O3,O4,O1,O2,H10'],
           } 

# Check burial of tail
# edit manually
# format: ligand_name: [check_burial_atom, linker_added_ligand_params,burial_cutoff (adjust this),
#                     lig_heavy_atm,lig_aromatic_amds]
ligands_tails = {
                 'T44':['C16',
                        '/home/linnaan/ligands/T44/sel_low_energy/T44_0005_relax_charged_ref2015_linker.params',
                        7,(),
                        ()],
                } #add tail ligands

In [16]:
# this contains 1 scaffold from each cluster, no filter on scaffold quality.
# can change with more knowledge
# pseudocycle lists
# ref: https://doi.org/10.1038/s41594-023-01112-6
#      https://github.com/LAnAlchemist/Psedocycles_NSMB.git
scaffold_df = pd.read_csv('finalist_21k_info_v220106_clustered_hullpos_forDock_cleaned_155aa.csv')
print(len(scaffold_df))
scaffold_df.head()

8719


Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  Unnamed: 0.1.1.1  \
0           0             0               2                 2   
1           1             1               4                 4   
2           2             2               5                 5   
3           3             3               7                 7   
4           4             4               8                 8   

   Unnamed: 0.1.1.1.1  Unnamed: 0.1.1.1.1.1    score   fa_atr   fa_rep  \
0                   2                     2 -305.763 -948.172  155.222   
1                   4                     4 -253.208 -880.858  121.295   
2                   5                     5 -245.363 -776.541   89.400   
3                   7                     7 -188.014 -515.873   76.572   
4                   8                     8 -239.014 -492.766   76.636   

    fa_sol  ...  \
0  715.014  ...   
1  707.958  ...   
2  650.947  ...   
3  352.040  ...   
4  367.920  ...   

                                                                                                                 name_before_permute  \
0   L_50_R_3_M_4_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211031_192628_612107_AAA+_step_00469_dldesign_5_relax_0001_rank_1_relax.pdb   
1  L_36_R_4_M_4_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211024_005219_391887_AAAA+_step_00492_dldesign_2_relax_0001_rank_1_relax.pdb   
2  L_32_R_4_M_3_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211023_233141_820550_AAAA+_step_00496_dldesign_1_relax_0001_rank_1_relax.pdb   
3   L_30_R_3_M_5_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211022_230901_908256_AAA+_step_00497_dldesign_4_relax_0001_rank_1_relax.pdb   
4  L_24_R_4_M_3_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211024_154223_597527_AAAA+_step_00446_dldesign_2_relax_0001_rank_1_relax.pdb   

              dssp_bin_before_permute                cluster  \
0        3/LHLHLHLHLHLHL_3/files.list     3/HHHHHH_3/62.list   
1  4/LHLHLHLHLHLHLHLHLHL_4/files.list   4/HHHHHHHH_4/21.list   
2    4/LHLHLHLHLHLHLHLHL_4/files.list  4/HHHHHHHH_4/735.list   
3        3/LELELELELELEL_3/files.list    3/EEEEEE_3/197.list   
4  4/LELELELELELELELELEL_4/files.list    4/EEEEEEE_4/11.list   

                                                                                                                                                                            pdb1  \
0                                               L_50_R_3_M_4_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211031_192628_612107_AAA+_step_00469_dldesign_5_relax_0001_rank_1_relax.pdb   
1  L_36_R_4_M_4_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211024_005219_391887_AAAA+_step_00492_dldesign_2_relax_0001_rank_1_relax_20220103_171107_923767_permute_rank_1_relax.pdb   
2  L_32_R_4_M_3_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211023_233141_820550_AAAA+_step_00496_dldesign_1_relax_0001_rank_1_relax_20220103_171105_507088_permute_rank_1_relax.pdb   
3                                               L_30_R_3_M_5_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211022_230901_908256_AAA+_step_00497_dldesign_4_relax_0001_rank_1_relax.pdb   
4                                              L_24_R_4_M_3_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211024_154223_597527_AAAA+_step_00446_dldesign_2_relax_0001_rank_1_relax.pdb   

                                                                                                                                                                                                                                posfile_by_hull  \
0                                               /home/drhicks1/af2_cyclic_repeat_proteins/20220119_pocket_hull/L_50_R_3_M_4_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211031_192628_612107_AAA+_step_00469_dldesign_5_relax_0001_rank_1_relax.pos   
1  /home/drhicks1/af2_cyclic_repeat_proteins/20220119_pocket_hull/L_36_R_4_M_4_MR_5_MM_uniform_T_0_02_LW_4_0_2_0_1_0_20211024_005219_391887_AAAA+_step_00492_dldesign_2_relax_0001_rank_1_relax_20220103_171107_923767_permute_rank_1_relax.pos   
2  /home/drhicks1/af2_cyclic_repeat_prot

In [5]:
def generate_hb_filters(atm_list,scorefxn):
    filters,protocols = [],[]
    for atm in atm_list:
        filters.append(f'<SimpleHbondsToAtomFilter name="hb_to_{atm}" n_partners="1" hb_e_cutoff="-0.3" target_atom_name="{atm}" res_num="%%ligand_res_number%%" scorefxn="{scorefxn}" confidence="0"/>')
        protocols.append(f'<Add filter="hb_to_{atm}"/>')
    return filters,protocols

In [6]:
def plot2pandas(df1,df2,features,legend1='sel',legend2='all',ncols=3):
    nrows = math.ceil(len(features) / ncols)
    (fig, axs) = plt.subplots(ncols=ncols, nrows=nrows, figsize=[4*ncols,3*nrows])
    axs = axs.reshape(-1)
    for (i, metric) in enumerate(features):
        sns.distplot(df1[metric].dropna(), ax=axs[i],label=legend1).legend()
        sns.distplot(df2[metric].dropna(), ax=axs[i],label=legend2).legend()
    sns.despine()
    plt.tight_layout()
    plt.show()

## 1.RIFgen

In [7]:
rifgen_dir = f'{scratch_dir}1_rifgen/'
os.makedirs(rifgen_dir,exist_ok=True)

In [8]:
for n,lig in enumerate(ligands):
    if (lig == 'T44'):
        os.makedirs(f'{rifgen_dir}{lig}_rifgen/',exist_ok=True)
         # change here to your RIFGen path
        temp_requirements = './toolkits/require_hb.txt'
        dst_requirements = f'{rifgen_dir}{lig}_rifgen/require_hb.txt'
        copyfile(temp_requirements,dst_requirements)
        hbond_requirements = [f'{n+1} HBOND {atm} 1' for n,atm in enumerate(ligands[lig][4].split(','))]
        hbond_requirements = '\n'.join(hbond_requirements)
        sed_inplace(dst_requirements, '__REQUIREMENTS__', hbond_requirements)

- Manual change `require_hb.txt` file for changing requirements

In [9]:
# Make directories
rifgen_dir = f'{scratch_dir}1_rifgen/'
os.makedirs(rifgen_dir,exist_ok=True)
for n,lig in enumerate(ligands):
    if (lig == 'T44'):
        os.makedirs(f'{rifgen_dir}{lig}_rifgen/',exist_ok=True)
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            rotamer_rifgen_dir = f'{rifgen_dir}{lig}_rifgen/{rotamer_name}/'
            os.makedirs(rotamer_rifgen_dir,exist_ok=True)
            os.makedirs(rotamer_rifgen_dir+'/cache',exist_ok=True)
            temp_requirements = f'{rifgen_dir}{lig}_rifgen/require_hb.txt'
            temp_flag = '1st_SC_screen/rifgen_no_req.flags_substitutable'
            # change temp_qsub to your own submission file
            temp_qsub = '1st_SC_screen/qsub.sh'
            dst_requirements = f'{rotamer_rifgen_dir}require_hb.txt'
            dst_flag = f'{rotamer_rifgen_dir}rifgen.flags'
            dst_qsub = f'{rotamer_rifgen_dir}qsub.sh'
            copyfile(temp_flag,dst_flag)
            sed_inplace(dst_flag, '__OUTDIR__', f'{rotamer_rifgen_dir}output')
            sed_inplace(dst_flag, '__PDB__', rotamer)
            sed_inplace(dst_flag, '__PARAMS__', ligands[lig][1]) #use ref2015params
            sed_inplace(dst_flag, '__ligand3letter__', lig)
            # add this if your computing system has specific locations for cache, need 1-2 G
            # by default, we create individual cache for each dock, it will take a lot of space, but
            # it prevents one failed job fail other jobs.
            #sed_inplace(dst_flag, '__LIG_CACHE__', f'{rifgen_dir}{lig}_rifgen/cache_dir')
            copyfile(temp_qsub,dst_qsub)
            copyfile(temp_requirements,dst_requirements)

In [ ]:
# test, if good, submit. You should get a few Gb of rif output/ for each rotamer.
for lig in ligands:
    if (lig == 'T44'): # or (lig == 'DIG')
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            rotamer_rifgen_dir = f'{rifgen_dir}{lig}_rifgen/{rotamer_name}/'
            if os.path.isfile(rotamer_rifgen_dir+'rifgen.log') == False:
                os.chdir(rotamer_rifgen_dir)
                subprocess.check_call(f'sbatch qsub.sh',shell=True)
                print(f'submit {rotamer_name}')

## 2. RIFdock

In [7]:
## Did not turn off Ala for rifdock. Found not important at scaffold selection stage.
rifdock_dir = f'{scratch_dir}2_rifdock/'
os.makedirs(rifdock_dir,exist_ok=True)

In [ ]:
# generate individual flags
for n,lig in enumerate(ligands):
    if (lig=='T44'):
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            indi_rot_dock_dir = f'{rifdock_dir}{lig}_rifdock/{rotamer_name}/'
            os.makedirs(f'{rifdock_dir}{lig}_rifdock/cache',exist_ok=True) #not needed. Rifdock create cache
            os.makedirs(indi_rot_dock_dir,exist_ok=True)
            temp_flag = '1st_SC_screen/rifdock_tail.flags'
            dst_flag = f'{indi_rot_dock_dir}rifdock.flags'
            copyfile(temp_flag,dst_flag)
            sed_inplace(dst_flag, '__PDBOUT__', '10')
            sed_inplace(dst_flag, '__REQUIREMENTS__', ligands[lig][3])
            sed_inplace(dst_flag, '__SCORECUTOFF__', str(int(0*(len(ligands[lig][3].split(','))))))
            rifgen_log = f'{scratch_dir}1_rifgen/{lig}_rifgen/{rotamer_name}/rifgen.log'
            sed_inplace(dst_flag, '__RIFGENOUT__', get_flags(rifgen_log,False))
            sed_inplace(dst_flag, '__CACHE_DIR__', f'{rifdock_dir}{lig}_rifdock/cache')
            temp_qsub = '1st_SC_screen/rifdock_qsub.sh'
            dst_qsub = f'{indi_rot_dock_dir}qsub.sh'
            copyfile(temp_qsub,dst_qsub)
# Manually change each flags if needed

In [ ]:
# Generate docking commands
for n,lig in enumerate(ligands):
    if (lig=='T44'):
        cmds=[]
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            indi_rot_dock_dir = f'{rifdock_dir}{lig}_rifdock/{rotamer_name}/'
            for i,(j,row) in enumerate(scaffold_df.iterrows()):
                if i >= 0:
                    scaffold = row['path']
                    cluster = row['cluster']
                    indi_cluster_dir = indi_rot_dock_dir + '/'.join(cluster.split('/')[0:2]) + \
                                       '_' + cluster.split('/')[2].replace('.list','') + '/'
                    os.makedirs(indi_cluster_dir,exist_ok=True)
                    os.chdir(indi_cluster_dir)
                    temp_flag = f'{rifdock_dir}{lig}_rifdock/{rotamer_name}/rifdock.flags'
                    copyfile(temp_flag,'rifdock.flags')
                    temp_flag = f'{rifdock_dir}{lig}_rifdock/{rotamer_name}/qsub.sh'
                    copyfile(temp_flag,'qsub.sh')
                    fp = open('SCAFFOLD.list','w')
                    fp.write(scaffold)
                    fp.close()
                    fp = open('POSFILE.list','w')
                    fp.write(home_dir+row['posfile_by_hull']) # you may need to change this based on how your scripts is set
                    fp.close()
                    if len(glob.glob(f'{indi_cluster_dir}out/*all.dok*')) == 0:
                        cmds.append(f'cd {indi_cluster_dir}; bash qsub.sh')
        fp = open(f'{rifdock_dir}{lig}_rifdock/{lig}_1strifdock_PC','w')
        fp.write('\n'.join(cmds))
        fp.close()

In [ ]:
# submit
for n,lig in enumerate(ligands):
    if (lig=='T44'):
        indi_dock_dir = f'{rifdock_dir}{lig}_rifdock/'
        os.chdir(indi_dock_dir)
        cmd_f = f'{lig}_1strifdock_PC'
        make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}.submit.sh',
            time='5:00:00',group_size=80,logsfolder='logs',cpus=2,mem='20g')
        subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)

In [ ]:
# count docks
docks = {}
total_silents = 0
for n,lig in enumerate(ligands):
    if (lig=='T44'):
        docks[lig]={}
        lig_rifdock_dir = f'{rifdock_dir}{lig}_rifdock/'
        for rotamer in ligands[lig][2]:
            docks_num = 0
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            docks[lig][rotamer_name] =[]
            success_cluster = 0
            indi_rot_dock_dir = f'{lig_rifdock_dir}{rotamer_name}/'
            for i,(j,row) in enumerate(scaffold_df.iterrows()):
                if i >= 0:
                    cluster = row['cluster']
                    indi_cluster_dir = indi_rot_dock_dir + row['indi_cluster_dir']
                    all_doks = glob.glob(f'{indi_cluster_dir}out/*all.dok*')
                    if len(all_doks) >= 0:
                        if len(glob.glob(f'{indi_cluster_dir}out/*.silent')) > 0:
                            success_cluster += 1
                            for all_dok in all_doks:
                                with open(all_dok,'r') as f: 
                                    docks_num += len(f.readlines())
                            docks[lig][rotamer_name] += glob.glob(f'{indi_cluster_dir}out/*.silent')
            print(f'{lig} {rotamer_name} got docks: {docks_num} from {success_cluster} clusters\n__________________')
            total_silents += success_cluster
        with open(f"{lig_rifdock_dir}dock_list.json", "w") as outfile:
            json.dump(docks[lig], outfile)
        print(f'{lig} generated {total_silents} stotal_silents in total')

In [ ]:
## !!!Remove cache direcotry and logs directory, they take lots of spaces

- (optional) Can write checkpoint file at rifdock directory to control which part of clusters go into design. Useful if just want to test some cluster for design or do not want to repeat design for some clusters
`checkpoint_f = f'{scratch_dir}rifdock/{lig}_rifdock/{rotamer_name}/already_docked_checkpoint'`

## 3_1. Predictor1

##### Use quick predictor (method1) to design all docks

In [ ]:
# prepare local version of flag and xml
tmp_protocol = '1st_SC_screen/FastPredictor_v2_talaris_ligand_3.xml'
tmp_flag = '1st_SC_screen/flags_rescore_talaris_1'
os.makedirs(f'{scratch_dir}3_prediction/',exist_ok=True)
for n,lig in enumerate(ligands):
    if (lig=='T44'):
        design_dir = f'{scratch_dir}3_prediction/{lig}_prediction/'
        os.makedirs(design_dir,exist_ok=True)
        dst_protocol = design_dir + f'FastPredictor_v2_talaris_ligand_3.xml'
        copyfile(tmp_protocol,dst_protocol)
        fiters,protocols = generate_hb_filters(ligands[lig][4].split(','),'sfxn')
#         sed_inplace(dst_protocol,'__HBFILTER__','\n'.join(fiters))
#         sed_inplace(dst_protocol,'__HBFILTERPROTOCOL__','\n'.join(protocols))
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            indi_rot_design_dir = f'{design_dir}{rotamer_name}/'
            os.makedirs(indi_rot_design_dir,exist_ok=True)
            dst_flag = indi_rot_design_dir + 'flags_rescore_talaris_1'
            copyfile(tmp_flag,dst_flag)
            sed_inplace(dst_flag,'__PARAMS__',ligands[lig][1]) #still use ref2015 since we are using talaris
#             sed_inplace(dst_flag,'__SUFFIX__',f'_{rotamer_name}')
            indi_dst_protocol = indi_rot_design_dir + f'FastPredictor_v2_talaris_ligand_3.xml'
            copyfile(dst_protocol,indi_dst_protocol)
# make changes to local version of flag and dock xml if needed.

In [ ]:
# cmd generation option 1: generate using Jupyter Notebook
TEST = False # if test, deposite PDB, if not, score only
# Generate individual design commands
ROSETTA = '/software/rosetta/latest/bin/rosetta_scripts.hdf5.linuxgccrelease' # change to your rosetta path
for n,lig in enumerate(ligands):
    if (lig=='T44'):
        design_dir = f'{scratch_dir}3_prediction/{lig}_prediction/'
        os.makedirs(design_dir,exist_ok=True)
        cmds = []
        os.chdir(design_dir)
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            print(f'For {rotamer_name}:________________________________')
            already_designed = []
            checkpoint_f = f'{scratch_dir}2_rifdock/{lig}_rifdock/{rotamer_name}/already_docked_checkpoint'
            if os.path.isfile(checkpoint_f):
                fp = open(checkpoint_f,'r')
                for line in fp:
                    already_designed.append(line.rstrip())
                fp.close()
            print(f'already designed {len(already_designed)}')
            protocol = 'FastPredictor_v2_talaris_ligand_3.xml'
            flag = 'flags_rescore_talaris_1'
            for n,dock in enumerate(docks[lig][rotamer_name]):
                if n >= 0:
                    indi_cluster_dir = f'{rotamer_name}/'+ dock.split('/')[-4]+\
                                       '/'+dock.split('/')[-3] + '/'
                    cluster_name = '_'.join(('/'.join(dock.split('/')[-4:-2])).split('_')[:-1])+'/'+\
                                 dock.split('/')[-3].split('_')[-1]+'.list'
                    if cluster_name not in already_designed:
                        os.makedirs(indi_cluster_dir,exist_ok=True)
                        lig_num = int(scaffold_df.loc[scaffold_df['cluster']==cluster_name,'nres'].values[0])+1
                        if TEST: 
                            if n < 1:
                                cmds.append(f'cd {design_dir}{indi_cluster_dir}; {ROSETTA} -parser:protocol ../../../{protocol} '+\
                                            f'@../../{flag} -in:file:silent {dock} ' + \
        #                                     f'-out:file:write_pdb_parametric_info True ' + \
#                                             f'-out:file:score_only ' +\
                                            f'-parser:script_vars ligand_res_number={lig_num}')
                            else:
                                break
                        else:
                            cmds.append(f'cd {design_dir}{indi_cluster_dir}; {ROSETTA} -parser:protocol ../../../{protocol} '+\
                                        f'@../../{flag} -in:file:silent {dock} ' + \
                                        f'-out:file:score_only predictor.sc ' +\
                                        f'-parser:script_vars ligand_res_number={lig_num}')
        cmd_f = f'{lig}_1stpredictor1'
        with open(cmd_f,'w') as f:
            f.write('\n'.join(cmds))
        print(f'write {len(cmds)} design commands')
        SUBMIT = False
        make_submit_file(cmds=cmd_f,
                submitfile=f'{cmd_f}.submit.sh',time='3:00:00',group_size=500,logsfolder='logs',cpus=1,
                mem='1g') ## SSpred usage save cache which will create error if visited too often. 
        if SUBMIT:
            subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)

##### What's most optimal allocation of each step?
- rifdock: 20gb, 2CPU
- predictor: 1 gb, 1cpu
- design: 3gb. 1cpu

In [ ]:
## Collect results
designs = {}
for n,lig in enumerate(ligands):
    if (lig == 'T44'):
        designs[lig]={}
        design_dir = f'{scratch_dir}3_prediction/{lig}_prediction/'
        total_num = 0
        for rotamer in ligands[lig][2]:
            success = 0
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            indi_rot_design_dir = f'{design_dir}{rotamer_name}/'
            designs[lig][rotamer_name] =[]
            for i,(j,row) in enumerate(scaffold_df.iterrows()):
                if i >= 0:
                    cluster =row['cluster']
                    indi_cluster_dir = indi_rot_design_dir + row['indi_cluster_dir']
                    predictor_sc_f = glob.glob(f'{indi_cluster_dir}predictor.sc')
                    if len(predictor_sc_f) > 0:
                        success += 1
                        designs[lig][rotamer_name] += predictor_sc_f
            print(f'{lig} {rotamer_name} from {success} clusters got designs: {len(designs[lig][rotamer_name])}\n__________________')
            total_num += len(designs[lig][rotamer_name])
        print(f'[SUM] {lig} got predictor scs in total: {total_num}')

In [ ]:
for n,lig in enumerate(ligands):
    if (lig == 'T44'):
        df_list = []
        design_dir = f'{scratch_dir}3_prediction/{lig}_prediction/'
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            for i,sc in enumerate(designs[lig][rotamer_name]):
                if i >= 0:
                    indi_df = pd.read_csv(sc,header=1,delim_whitespace=True)
                    indi_df['indi_cluster_dir'] = '/'.join(sc.split('/')[-3:-1])+'/'
                    indi_cluster_dir = '/'.join(sc.split('/')[-3:-1])+'/'
                    indi_df['cluster'] = scaffold_df.loc[scaffold_df['indi_cluster_dir']==indi_cluster_dir,'cluster'].values[0]
                    indi_df['rotamer'] = rotamer_name
                    silent_in = f'{scratch_dir}2_rifdock/{lig}_rifdock/{rotamer_name}/{indi_cluster_dir}out/'+\
                                indi_df.description[0].split('_000000')[0]+'.silent'
                    if not os.path.isfile(silent_in):
                        print(f'Error {silent_in}')
                    indi_df['silent_in'] = silent_in
                    
                    df_list.append(indi_df)
        all_df = pd.concat(df_list)
        all_df['name'] = all_df['description'] + '_' + all_df['rotamer']
        all_df.to_csv(f'{design_dir}all_predictor.sc')
        print(f'{lig} {len(all_df)}') 
all_df

In [ ]:
finalized = True#use figure and numbers to decide the CMS cutoff to use
CMS_CUTOFF = {'T44':50/100} #percentile
ncols = 3  
nrows = math.ceil(len(ligands) / ncols) #because we will also be plotting ROCs  
(fig, axs) = plt.subplots(ncols=ncols, nrows=nrows, figsize=[4*ncols,4*nrows]  )  
axs = axs.reshape(-1)
for n,lig in enumerate(ligands):
    if (lig == 'T44'): #(lig == 'APC') or (lig == 'HCY') or 
        design_dir = f'{scratch_dir}3_prediction/{lig}_prediction/'
        all_df = pd.read_csv(f'{design_dir}all_predictor.sc')
        all_df = all_df.sort_values(by='contact_molecular_surface',ascending=False)
        value = all_df['contact_molecular_surface'].quantile(q=CMS_CUTOFF[lig],interpolation="nearest")
        sel_df = all_df[all_df['contact_molecular_surface']>=value]
        print(f'{lig} CMS at quantile {CMS_CUTOFF[lig]} has {value}, {len(sel_df)}/{len(all_df)} docks selected')
        sns.histplot(data=all_df,x='contact_molecular_surface',label='all',color='blue',ax=axs[n]).set_xlabel(f'{lig} contact_molecular_surface')
        sns.histplot(data=sel_df,x='contact_molecular_surface',label='sel',color='orange',ax=axs[n]).set_xlabel(f'{lig} contact_molecular_surface')
        axs[n].legend()
        if finalized:
            sel_df.to_csv(f'{design_dir}sel_for_2ndpredictor.sc')

In [ ]:
# optional: extract chosen silent files and remove bad ones, to save space.
SUBMIT = True
for n,lig in enumerate(ligands):
    if (lig == 'T44'): # or (lig == 'EST'): #(lig == 'APC') or (lig == 'HCY')
        design_dir = f'{scratch_dir}3_prediction/{lig}_prediction/'
        os.chdir(design_dir)
        sel_df = pd.read_csv(f'{design_dir}sel_for_2ndpredictor.sc')
        cmds = []
        for i,silent_f in enumerate(set(list(sel_df['silent_in']))):
            if i>=0:
                sel_cluster = sel_df[sel_df['silent_in']==silent_f]
                tags = ' '.join(list(sel_cluster['description']))
                if len(tags) > 0:
                    new_silent_f = silent_f.replace('.silent','_sel.silent')
                    cmds.append(f'echo "{tags}" | silentslice {silent_f} > {new_silent_f}')
        cmd_f = f'{lig}_extract_silent'
        with open(cmd_f,'w') as fp:
            fp.write('\n'.join(cmds))
        make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}_submit.sh',time='3:00:00',
                        group_size=int(len(cmds)/200)+1,logsfolder='logs',cpus=1,mem='1g')
        if SUBMIT:
            subprocess.check_call(f'sbatch {cmd_f}_submit.sh',shell=True)

In [ ]:
# remove original rifdock silent file to save space
for n,lig in enumerate(ligands):
    if (lig == 'T44'):
        start_time = time.time()
        cmds = []
        design_dir = f'{scratch_dir}2_rifdock/{lig}_rifdock/'
        os.chdir(design_dir)
        if os.path.isfile(f'{scratch_dir}/3_prediction/{lig}_prediction/sel_for_2ndpredictor.sc'):
            with open('dock_list.json','r') as fp:
                dock_dic = json.load(fp)
            for rotamer_name in dock_dic:
                cmds += [f'rm {silent_f}' for silent_f in dock_dic[rotamer_name]]
            cmd_f = f'rm_{lig}_orig_silent'
            with open(cmd_f,'w') as fp:
                fp.write('\n'.join(cmds))
            make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}.submit.sh',time="12:00:00",
                             group_size=int(len(cmds)/10),
            logsfolder='logs',cpus=1,mem='1g',)
            print(f'wrote for {lig}')
            SUBMIT = False
            if SUBMIT:
                subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)

## 3_2. Slower predictor to continue predict: old broken beta_nov16

In [ ]:
# prepare local version of flag and xml
tmp_protocol = '1st_SC_screen/LA_quick_design_select_dock_genpot_2.xml'
tmp_flag = '1st_SC_screen/flags_rescore_genpot_1'
os.makedirs(f'{scratch_dir}3_prediction_ref2015_oldmethod/',exist_ok=True)
for n,lig in enumerate(ligands):
    if (lig == 'T44'):# or (lig == 'HCY'):
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        os.makedirs(design_dir,exist_ok=True)
        dst_protocol = design_dir + f'LA_quick_design_select_dock_genpot_2.xml'
        copyfile(tmp_protocol,dst_protocol)
        fiters,protocols = generate_hb_filters(ligands[lig][4].split(','),'sfxn')
        sed_inplace(dst_protocol,'__HBFILTER__','\n'.join(fiters))
        sed_inplace(dst_protocol,'__HBFILTERPROTOCOL__','\n'.join(protocols))
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            indi_rot_design_dir = f'{design_dir}{rotamer_name}/'
            os.makedirs(indi_rot_design_dir,exist_ok=True)
            dst_flag = indi_rot_design_dir + 'flags_rescore_genpot_1'
            copyfile(tmp_flag,dst_flag)
            sed_inplace(dst_flag,'__PARAMS__',ligands[lig][1]) #still use ref2015 since we are using talaris
#             sed_inplace(dst_flag,'__SUFFIX__',f'_{rotamer_name}')
            indi_dst_protocol = indi_rot_design_dir + f'LA_quick_design_select_dock_genpot_2.xml'
            copyfile(design_dir + f'LA_quick_design_select_dock_genpot_2.xml',indi_dst_protocol)
# make changes to local version of flag and dock xml if needed.

In [ ]:
# Generate individual design commands
# change to your Rosetta path
ROSETTA = '/software/rosetta/latest/bin/rosetta_scripts.hdf5.linuxgccrelease'
for n,lig in enumerate(ligands):
    if (lig == 'T44'):# or (lig == 'HCY'):
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        sel_df = pd.read_csv(f'{scratch_dir}3_prediction/{lig}_prediction/sel_for_2ndpredictor.sc')
        cmds = []
        os.chdir(design_dir)
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            print(f'For {rotamer_name}:________________________________')
            protocol = 'LA_quick_design_select_dock_genpot_2.xml'
            flag = 'flags_rescore_genpot_1'
            rotamer_sel_df = sel_df[sel_df['rotamer']==rotamer_name]
            for n,silent_in in enumerate(list(set(rotamer_sel_df.silent_in))):
                if n >= 0:
                    indi_silent_df = rotamer_sel_df[rotamer_sel_df['silent_in']==silent_in]
                    for indi_cluster_dir in list(set(indi_silent_df.indi_cluster_dir)):
                        lig_num = 1+int(scaffold_df.loc[scaffold_df['indi_cluster_dir']==indi_cluster_dir,'nres'].values[0])
                        indi_cluster_dir = f'{rotamer_name}/'+ indi_cluster_dir
                        os.makedirs(indi_cluster_dir,exist_ok=True)
                        sel_silent_in = silent_in.replace('.silent','_sel.silent')
                        cmds.append(f'cd {design_dir}{indi_cluster_dir}; {ROSETTA} -parser:protocol '+\
                                    f'../../../{protocol} @../../{flag} -in:file:silent '+\
                                    f'{sel_silent_in} ' + \
                                    f'-out:file:score_only predictor.sc ' +\
                                    f'-parser:script_vars ligand_res_number={lig_num}')
        cmd_f = design_dir + f'{lig}_predictor2_cmds'
        with open(cmd_f,'w') as fp:
            fp.write('\n'.join(cmds))
        make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}.submit.sh',time="3:00:00",
                group_size=50 ,logsfolder='logs',cpus=1,mem='1g',limit_job_to100=False,
                needs_gpu=False) ## SSpred usage save cache which will create error if visited too often. 
        print(f'write {len(cmds)} design commands')
        SUBMIT = True
        if SUBMIT:
            subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)

In [ ]:
## Collect results
designs = {}
for n,lig in enumerate(ligands):
    if (lig == 'T44'): #or (lig == 'HCY'):
        designs[lig]={}
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        for rotamer in ligands[lig][2]:
            success = 0
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            indi_rot_design_dir = f'{design_dir}{rotamer_name}/'
            designs[lig][rotamer_name] =[]
            for i,(j,row) in enumerate(scaffold_df.iterrows()):
                if i >= 0:
                    cluster = row['cluster']
                    indi_cluster_dir = indi_rot_design_dir + '/'.join(cluster.split('/')[0:2]) + \
                                       '_' + cluster.split('/')[2].replace('.list','') + '/'
                    predictor_sc_f = glob.glob(f'{indi_cluster_dir}predictor.sc')
                    if len(predictor_sc_f) > 0:
                        success += 1
                        designs[lig][rotamer_name] += predictor_sc_f
            print(f'{lig} {rotamer_name} from {success} clusters got designs: {len(designs[lig][rotamer_name])}\n__________________')

In [ ]:
for n,lig in enumerate(ligands):
    if (lig == 'T44'):# or (lig == 'HCY'):
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        df_list = []
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            for sc in designs[lig][rotamer_name]:
                indi_df = pd.read_csv(sc,header=1,delim_whitespace=True)
                indi_cluster_dir = '/'.join(sc.split('/')[-3:-1])+'/'
                indi_df['indi_cluster_dir'] = indi_cluster_dir
                indi_df['cluster'] = scaffold_df.loc[scaffold_df['indi_cluster_dir']==indi_cluster_dir,'cluster'].values[0]
                indi_df['rotamer'] = rotamer_name
                silent_in = f'{scratch_dir}2_rifdock/{lig}_rifdock/{rotamer_name}/{indi_cluster_dir}out/'+\
                                indi_df.description[0].split('_000000')[0]+'.silent'
                if not os.path.isfile(silent_in):
                    print(f'Error {silent_in}')
                indi_df['silent_in'] = silent_in
                df_list.append(indi_df)
        all_df = pd.concat(df_list)
        all_df['name'] = all_df['description'] + '_' + all_df['rotamer']
        all_df.to_csv(f'{design_dir}all_predictor.sc')
all_df

#### Check burial of tail
- Manually add tail and check tail burail
- From Gyu Rie 
- `/home/linnaan/from/gyurie/ligand_linker_orientation_check`
1. Generate a params file with an extended linker that would mimic some part of the conjugation linker (doesn't have to be accurate). It will be 'packed' later.
@add_linker_params/
#You would need a ligand pdb ('hcy.pdb') and a molecule pdb ('linker_newname.pdb' can be used for some cases) to align at least three atoms and add the linker.
example:
`sh ./cmd`
The script will add additional atoms and modify the params file to add proton_chi lines.
example output-> hcy_linker_tors.params

2. We will use the params file generated from 1. to run a fast packer and calculate atomic depth for the linker atoms. (If the atoms that consist the linker are buried we can filter those out). The modified params file will be used to automatically fill in the missing atoms (linker atoms), so no need to change the input pdb (your design pdb). You just need to use the modified params.
script: check_burial.py

In [ ]:
CHECK_BURIAL = '/software/conda/envs/pyrosetta/bin/python '+\ # you should be fine as long as you use pyrosetta
               'toolkits/check_burial.py'
for n,lig in enumerate(ligands):
    if (lig == 'T44'):# or (lig == 'HCY'):
        cmds = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        sel_df = pd.read_csv(f'{scratch_dir}3_prediction/{lig}_prediction/sel_for_2ndpredictor.sc')
        os.chdir(design_dir)
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            print(f'For {rotamer_name}:________________________________')
            rotamer_sel_df = sel_df[sel_df['rotamer']==rotamer_name]
            for i,silent_in in enumerate(list(set(rotamer_sel_df.silent_in))):
                if i>= 0:
                    indi_silent_df = rotamer_sel_df[rotamer_sel_df['silent_in']==silent_in]
                    for indi_cluster_dir in list(set(indi_silent_df.indi_cluster_dir)):
                        lig_num = 1+int(scaffold_df.loc[scaffold_df['indi_cluster_dir']==indi_cluster_dir,'nres'].values[0])
                        indi_cluster_dir = f'{rotamer_name}/'+ indi_cluster_dir
                        os.makedirs(indi_cluster_dir,exist_ok=True)
                        sel_silent_in = silent_in.replace('.silent','_sel.silent')
                        tags = ','.join(list(indi_silent_df['description']))
                        cmds.append(f'{CHECK_BURIAL} --silent {sel_silent_in} --lig_atom_names '+\
                              ligands_tails[lig][0] + f' --tags {tags}'+\
                              f' --params_file '+ ligands_tails[lig][1] +\
                              f' --out_sc_file {indi_cluster_dir}'+\
                              os.path.basename(sel_silent_in).replace('.silent','_check_burial.sc'))
        cmd_f = f'{design_dir}{lig}_check_burial'
        with open(cmd_f,'w') as f:
            f.write('\n'.join(cmds))
        if len(cmds) > 0:
            make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}.submit.sh',time="3:00:00",
                                group_size=50,logsfolder='logs',cpus=1,mem='1g')
            SUBMIT = True
            if SUBMIT:
                subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)

In [ ]:
## Collect the burial scores
for n,lig in enumerate(ligands):
    if (lig == 'T44'):# or (lig == 'HCY'):
        df_list = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        chose_df = pd.read_csv(f'{design_dir}/all_predictor.sc')
        sc_fs = glob.glob(design_dir+'*/*/*/*_check_burial.sc')
        for sc in sc_fs:
            indi_df = pd.read_csv(sc,header=0,delim_whitespace=True)
            indi_df['rotamer'] = sc.split('/')[-4]
            indi_df['name'] = indi_df['description']+'_'+indi_df['rotamer']
            df_list.append(indi_df)
        burial_df = pd.concat(df_list)
        chose_df = chose_df.merge(burial_df[['name','total_atom_depth']],how='left',on='name')
        chose_df['total_atom_depth'] = chose_df['total_atom_depth'].astype(float)
        chose_df.to_csv(f'{design_dir}/all_predictor.sc')
        sel_chose_df = chose_df[chose_df['total_atom_depth'] <= ligands_tails[lig][2]]
        sel_chose_df.to_csv(f'{design_dir}chosen_scaffolds_to_extract.csv')
        print(f'{lig} from {len(chose_df)} select {len(sel_chose_df)}, scaffold '+str(len(set(sel_chose_df.cluster))))
sel_chose_df

## 3_3. (optional) Randomly select 1000 silent docks for real design to test if predictor reflects real scores
- only pick one tag to design

In [ ]:
random_docks = {}
for lig in ligands:
    if lig == 'T44':
        count = 0
        for rotamer in docks[lig]:
            if len(docks[lig]) > 0:
                count += 1
        print(count)
        random_docks[lig] = {}
        for rotamer in docks[lig]:
            if len(docks[lig][rotamer]) > int(1000/count)+1:
                random_docks[lig][rotamer] = random.sample(docks[lig][rotamer],int(1000/count))
            else:
                random_docks[lig][rotamer] = []
for lig in random_docks:
    if lig == 'T44':
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        sum_silent = 0
        for rotamer in random_docks[lig]:
            sum_silent += len(random_docks[lig][rotamer])
    print(f'selected {sum_silent} for {lig}')
    ## save random docks in a file for record
    json_obj = json.dumps(random_docks,indent=4)
    with open(f'{design_dir}sel_random_docks.json','w') as f:
        json.dump(random_docks,f)

In [ ]:
## test cmd
TEST = False
PYTHON = '/software/conda/envs/pyrosetta/bin/python'
SCRIPTS = '1st_SC_screen/design_ligand_full_noHBNet_1.py'
for n,lig in enumerate(ligands):
    if lig != 'T44':
        design_dir = f'{scratch_dir}3_prediction_real_design/{lig}_prediction/'
        os.makedirs(design_dir,exist_ok=True)
        cmds = []
        os.chdir(design_dir)
        sel_dock_json = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/sel_random_docks.json'
        with open(sel_dock_json,'r') as fp:
            random_docks = json.load(fp)
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            print(f'For {rotamer_name}:________________________________')
            for i,dock in enumerate(random_docks[lig][rotamer_name]):
                if i >= 0:
                    indi_cluster_dir = f'{rotamer_name}/'+ dock.split('/')[-4]+\
                                       '/'+dock.split('/')[-3] + '/'
                    cluster_name = '_'.join(('/'.join(dock.split('/')[-4:-2])).split('_')[:-1])+'/'+\
                                 dock.split('/')[-3].split('_')[-1]+'.list'
                    os.makedirs(indi_cluster_dir,exist_ok=True)
                    pdb = scaffold_df.loc[scaffold_df['cluster']==cluster_name,'path'].values[0]
                    lig_num = int(scaffold_df.loc[scaffold_df['cluster']==cluster_name,'nres'].values[0])+1
                    # just take the first one
                    tag = list(silent_tools.build_silent_index(dock)['index'].keys())[0]
                    # you may need to change here, depending on where you put the pssm files
                    pssm = home_dir + scaffold_df.loc[scaffold_df['cluster']==cluster_name,'polar_bias_pssm_f'].values[0]
                    cmds.append(f'cd {indi_cluster_dir}; {PYTHON} {SCRIPTS} '+\
                                f'--silent {dock} --params ' + ligands[lig][1] + ' ' + \
                                f'--heavy_atms ' + ligands[lig][4] + \
                                f' --tags {tag} --ligand_res_num {lig_num} '+\
                                f'--save_to_pdb True --pssm {pssm}')
        cmd_f = f'{design_dir}{lig}_real_cmds'
        with open(cmd_f,'w') as fp:
            fp.write('\n'.join(cmds))
        print(f'write {len(cmds)} design commands')
        make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}.submit.sh',queue='short',
                group_size=1,logsfolder='logs',cpus=1,mem='3g') 
        SUBMIT = True
        if SUBMIT:
            subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)

In [ ]:
## Collect results
designs = {}
for n,lig in enumerate(ligands):
    if n>=0:
        designs[lig]={}
        design_dir = f'{scratch_dir}3_prediction_real_design/{lig}_prediction/'
        for rotamer in ligands[lig][2]:
            success = 0
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            indi_rot_design_dir = f'{design_dir}{rotamer_name}/'
            designs[lig][rotamer_name] =[]
            for i,(j,row) in enumerate(scaffold_df.iterrows()):
                if i >= 0:
                    cluster = row['cluster']
                    indi_cluster_dir = indi_rot_design_dir + '/'.join(cluster.split('/')[0:2]) + \
                                       '_' + cluster.split('/')[2].replace('.list','') + '/'
                    if len(glob.glob(f'{indi_cluster_dir}*.sc')) > 0:
                        success += 1
                        designs[lig][rotamer_name] += (glob.glob(f'{indi_cluster_dir}*.sc'))
            print(f'{lig} {rotamer_name} from {success} '+\
                  f'clusters got designs: {len(designs[lig][rotamer_name])}\n__________________')

In [ ]:
for n,lig in enumerate(ligands):
    df_list = []
    if n>=0:
        design_dir = f'{scratch_dir}3_prediction_real_design/{lig}_prediction/'
        for rotamer in ligands[lig][2]:
            rotamer_name = os.path.basename(rotamer).replace('.pdb','')
            if len(designs[lig][rotamer_name]) > 0:
                for sc in designs[lig][rotamer_name]:
                    indi_df = pd.read_csv(sc)
                    indi_df['rotamer'] = rotamer_name
                    indi_df['name'] = indi_df['description'].str.split('.').str[0] + '_' + indi_df['rotamer']
                    df_list.append(indi_df)
        all_df = pd.concat(df_list)
        all_df = all_df.reset_index()
        all_df.to_csv(f'{design_dir}all_predictor.sc')
all_df

## 3_5. Plot ROC curve to pick cutoff to select docks for real designs

#### A) Quick predictor

In [ ]:
import imp
import libSlurm
imp.reload(libSlurm)
from libSlurm import plot_ROC

In [ ]:
# validate predictor1
pred1_FEATURE = 'contact_molecular_surface'
for n,lig in enumerate(ligands):
    if n>=0:
        print(f'predictor1 {lig}________')
        df_predictor1 = pd.read_csv(f'{scratch_dir}3_prediction/{lig}_prediction/all_predictor.sc')
        df_real = pd.read_csv(f'{scratch_dir}3_prediction_real_design/{lig}_prediction/all_predictor.sc')
        print(len(df_predictor1),len(df_real)) #len(df_MPNN),
        df_predictor1_has_real = df_predictor1[df_predictor1['name'].isin(list(df_real['name']))]
             
        df_feature = df_real[['name',pred1_FEATURE]]
        df_feature = df_feature.merge(df_predictor1_has_real[['name',pred1_FEATURE]],
                     how='left',on='name').rename(columns={'contact_molecular_surface_x':'contact_molecular_surface_real','contact_molecular_surface_y':'contact_molecular_surface_pred1'})
        df_feature = df_feature.dropna()
        print(len(df_feature))
        plot_ROC(df_feature,[pred1_FEATURE],[pred1_FEATURE],real_maker='_real',test_maker='_pred1')

In [ ]:
# validate predictor2
pred2_FEATURE = ['contact_molecular_surface','ddg2','holes_around_lig','total_hb_to_lig','dsasa']
merge_feature = ['contact_molecular_surface','ddg2','holes_around_lig','total_hb_to_lig','dsasa','name']
higher = ['contact_molecular_surface','total_hb_to_lig','dsasa']
for n,lig in enumerate(ligands):
    if n>=0:
        print(f'predictor2 {lig}________')
        df_predictor2 = pd.read_csv(f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/all_predictor.sc')
        df_real = pd.read_csv(f'{scratch_dir}3_prediction_real_design/{lig}_prediction/all_predictor.sc')
        df_predictor2['total_hb_to_lig'] = 0
        for col in list(df_predictor2.columns):
            if 'hb_to' in col:
                df_predictor2['total_hb_to_lig'] += df_predictor2[col]
        df_real['total_hb_to_lig'] = 0
        for col in list(df_real.columns):
            if 'hb_to' in col:
                df_real['total_hb_to_lig'] += df_real[col]
        df_predictor2 = df_predictor2.rename(columns={'ddg_norepack':'ddg2'})
        print(len(df_predictor2),len(df_real)) #len(df_MPNN),
        df_predictor2_has_real = df_predictor2[df_predictor2['name'].isin(list(df_real['name']))]
        
        df_feature = df_real[merge_feature]
        df_feature = df_feature.merge(df_predictor2_has_real[merge_feature],how='left',on='name')
        df_feature = df_feature.dropna()
        print(len(df_feature))
        for feature in pred2_FEATURE:
            df_feature = df_feature.rename(columns={f'{feature}_x':f'{feature}_real',f'{feature}_y':f'{feature}_pred2'})                      
        plot_ROC(df_feature,pred2_FEATURE,higher,real_maker='_real',test_maker='_pred2')

## 3_6. Select winner scaffolds

In [ ]:
## Collect results
CHECK_DESIGN = True
rifdock_dir = f'{scratch_dir}2_rifdock/'
quantiles = {
            'contact_molecular_surface':[False,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
             'dsasa':[False,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
             'ddg_norepack':[True,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
              'holes_around_lig':[True,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
             'total_atom_depth':[True,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]]
            } # select till quantiles are still represented based on ROC
for n,lig in enumerate(ligands):
    if (lig=='T44'): #or (lig=='HCY'):
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        predictor_sc = f'{design_dir}all_predictor.sc'
        check_design_dir = f'{design_dir}check_designs_at_quantiles'
        os.makedirs(check_design_dir,exist_ok=True)
        for feature in quantiles:
            predictor_df = pd.read_csv(predictor_sc)
            predictor_df['pdb_description'] = predictor_df['description'].str.split('_0000000').str[0]
            predictor_df = predictor_df.sort_values(by=feature,ascending=quantiles[feature][0])#.drop_duplicates(subset='cluster',keep='first')
            sns.histplot(data=predictor_df,x='rotamer')
            for quantile in quantiles[feature][1]:
                if quantiles[feature][0]:
                    value = predictor_df.quantile(q=(100-quantile)/100,axis=0,interpolation='nearest')[feature]
                    print('___________________________________________\n'+\
                         f'! at quantile {feature} {quantile}: {value}, '+\
                         f'scaffold left: {len(predictor_df[predictor_df[feature]<=value])}')
                else:
                    value = predictor_df.quantile(q=quantile/100,axis=0,interpolation='nearest')[feature]
                    print('___________________________________________\n'+\
                         f'! at quantile {feature} {quantile}: {value}, '+\
                         f'scaffold left: {len(predictor_df[predictor_df[feature]>=value])}')
                pdb_row = predictor_df[predictor_df[feature]==value]
                rotamer_name = pdb_row['rotamer'].values[0]
                cluster = pdb_row['cluster'].values[0]
                silent_f = f'{rifdock_dir}/{lig}_rifdock/{rotamer_name}/'+ cluster.split('/')[-3]+\
                               '/'+cluster.split('/')[-2] + '_' + \
                                cluster.split('/')[-1].replace('.list','') +\
                               '/out/' + pdb_row['description'].values[0].split('_0000000')[0] + '_sel.silent'
                tag = pdb_row['description'].values[0]
                if os.path.isfile(silent_f):
                    os.chdir(check_design_dir)
                    #this cmd need silent_tools
                    cmd = f'silentextractspecific {silent_f} {tag}; mv {tag[:-2]}*.pdb {feature}{quantile}.pdb'
                    subprocess.check_call(cmd,shell=True)
                    print(quantile,cmd,end='\n')

### look at the selected designs and check what's the quantile to choose.

In [ ]:
# Manually edit this
# feature:[quantile,ascending=T/F,corresponding_value] #should take top quantile
CHOSE_QUANTILE = {
                  'T44':{'contact_molecular_surface':[70,False,342],
                         'ddg_norepack':[15,True,-8.9],
                         'dsasa':[50,False,0.893],
                         'holes_around_lig':[15,True,3.47],
                         'total_atom_depth':[1,True,8]
                         },
                }

In [ ]:
finalized_SEL = True
for n,lig in enumerate(ligands):
    if (lig == 'T44'): #or (lig == 'HCY'):
        cmds = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        predictor_sc = f'{design_dir}chosen_scaffolds_to_extract.csv' #f'{design_dir}all_predictor.sc'
        predictor_df = pd.read_csv(predictor_sc)
        predictor_df['pdb_description'] = predictor_df['description'].str.split('_0000000').str[0]
        chose_df = predictor_df.copy()
        print(f'{lig}___________________________________')
        print(f'start from {len(chose_df)}')
        for feature in CHOSE_QUANTILE[lig]:
            chose_df = chose_df.sort_values(by=feature,ascending=CHOSE_QUANTILE[lig][feature][1])
            predictor_df = predictor_df.sort_values(by=feature,ascending=CHOSE_QUANTILE[lig][feature][1])
            value = CHOSE_QUANTILE[lig][feature][2]
            if CHOSE_QUANTILE[lig][feature][1]: #ascending=True
                chose_df = chose_df[chose_df[feature] <= value]
                print(f'{feature} at quantile {CHOSE_QUANTILE[lig][feature][0]} less than value {value},',
                      f'left design {len(chose_df)}')
            else:
                chose_df = chose_df[chose_df[feature] >= value]
                print(f'{feature} at quantile {CHOSE_QUANTILE[lig][feature][0]} greater than value {value},',
                      f'left design {len(chose_df)}')
        print(f'left designs '+str(len(chose_df)/len(predictor_df)*100)+'%')
        print(f'chose scaffolds '+str(len(set(list(chose_df['cluster'])))))
        #make_dist_plots(chose_df,CHOSE_QUANTILE[lig].keys())
        plot2pandas(chose_df,predictor_df,CHOSE_QUANTILE[lig].keys(),'chose','all',ncols = 4)
        
        if finalized_SEL:
            chose_df['orig_silent_f'] = f'{rifdock_dir}/{lig}_rifdock/'+chose_df['rotamer']+'/'+\
                                        chose_df['indi_cluster_dir']+'out/'+\
                                        chose_df['description'].str.split('_0000000').str[0]+'_sel.silent'
            chose_df.to_csv(f'{design_dir}chosen_scaffolds_to_check_burial.csv')
            #design these chosen docks for accurate MPNN resampling

## 3_7. Extract selected docks by PSSM-based rosetta design

In [ ]:
# extract pdbs. Previously we only dump the scores. To resample using MPNN, we need to use actual designed 
# pdbs. Thus we need to do quick design of chosen pdbs.
chose_designs = {}
PYTHON = '/software/conda/envs/pyrosetta/bin/python' # use pyrosetta should work
SCRIPTS = '1st_SC_screen/design_ligand_full_noHBNet_1.py'
for n,lig in enumerate(ligands):
    if (lig == 'T44'):# or (lig == 'HCY'):
        cmds = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        sel_chose_df = pd.read_csv(f'{design_dir}chosen_scaffolds_to_check_burial.csv')
        #design these chosen docks for accurate MPNN resampling
        chosen_design_dir = f'{design_dir}chosen_designs_to_resample/'
        for i,(j,row) in enumerate(sel_chose_df.iterrows()):
            if i >= 0:
                rotamer_name = row['rotamer']
                cluster = row['cluster']
                silent_f = row['silent_in']
                indi_cluster_dir = scaffold_df.loc[scaffold_df['cluster']==cluster,
                                                   'indi_cluster_dir'].values[0]
                # you may need to change here, depending on where you put your pssm
                pssm = home_dir + scaffold_df.loc[scaffold_df['cluster']==cluster,
                                                   'polar_bias_pssm_f'].values[0]
                lig_num = 1+int(scaffold_df.loc[scaffold_df['cluster']==cluster,'nres'].values[0])
                tag = row['description']
                os.makedirs(f'{chosen_design_dir}{rotamer_name}/{indi_cluster_dir}',exist_ok=True)
                end_pdb = f'{chosen_design_dir}{rotamer_name}/{indi_cluster_dir}{tag}_{rotamer_name}.pdb'
                if (os.path.isfile(silent_f) == True) and (os.path.isfile(end_pdb) == False):
                    cmds.append(f'cd {chosen_design_dir}{rotamer_name}/{indi_cluster_dir}; '+\
                                            f'{PYTHON} {SCRIPTS} '+\
                                            f'--silent {silent_f} --params ' + ligands[lig][1] + ' ' + \
                                            f'--heavy_atms ' + ligands[lig][4] + \
                                            f' --pssm {pssm} ' + \
                                            f' --tags {tag} --ligand_res_num {lig_num} '+\
                                            f'--save_to_pdb True --suffix _{rotamer_name}')

                else:
                    continue
                    #print(f'{silent_f} not there or already designed!')
        cmd_f = f'{design_dir}{lig}_extract_chosen_comp'
        with open(cmd_f,'w') as f:
            f.write('\n'.join(cmds))
        make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}.submit.sh',time='1:00:00',group_size=3,
                            logsfolder='logs',cpus=1,mem='2g')
        SUBMIT = True
        if SUBMIT:
            subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)


In [ ]:
#collect design results
for n,lig in enumerate(ligands):
    if lig == 'T44':
        df_list = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        sel_chose_df = pd.read_csv(f'{design_dir}chosen_scaffolds_to_extract.csv')
        sc_fs = glob.glob(f'{design_dir}chosen_designs_to_resample/*/*/*/*.sc')
        for sc in sc_fs:
            indi_df = pd.read_csv(sc)
            indi_df['pdb_path'] = sc.replace('.sc','.pdb')
            indi_df['indi_cluster_dir'] = '/'.join(sc.split('/')[-3:-1])+'/'
            indi_df['cluster'] = scaffold_df.loc[scaffold_df['indi_cluster_dir']=='/'.join(sc.split('/')[-3:-1])+'/','cluster'].values[0]
            df_list.append(indi_df)
        all_df = pd.concat(df_list)
        sel_chose_df['name'] = sel_chose_df['name']+'.pdb'
        all_df = all_df.merge(sel_chose_df[['name','total_atom_depth']],how='left',
                              left_on='description',right_on='name')
        all_df['group'] = 'original'
        all_df['rotamer'] = all_df['pdb_path'].str.split('/').str[-4]

        common_features = ['SC', 'ala_core_count', 'buns_all_heavy_ball',
       'buns_bb_heavy_ball', 'buns_sc_heavy_ball', 'cavity',
       'contact_molecular_surface', 'ddg2', 'dsasa', 'geometry', 'hb_lr_bb',
       'hb_lr_bb_per_res', 'hb_sr_bb', 'hb_sr_bb_per_res', 'hole',
       'holes_around_lig', 'hydrophobic_residue_contacts',
       'interface_buried_sasa', 'interface_sc', 'mismatch_probability', 'nALA',
       'nARG', 'nHIS', 'nMET', 'sap_A', 'sap_all', 'sbuns_all_heavy',
       'sbuns_all_heavy_no_ligand', 'score', 'score_per_res', 'sspred_overall',
       'timed', 'vbuns_all_heavy', 'vbuns_all_heavy_no_ligand', 'fa_atr', 'fa_rep', 
       'fa_sol', 'fa_intra_atr_xover4','fa_intra_rep_xover4', 'fa_intra_sol_xover4', 
       'lk_ball', 'lk_ball_iso','lk_ball_bridge', 'lk_ball_bridge_uncpl', 'fa_elec', 'fa_intra_elec',
       'pro_close', 'hbond_sr_bb', 'hbond_lr_bb', 'hbond_bb_sc', 'hbond_sc',
       'dslf_fa13', 'omega', 'fa_dun_dev', 'fa_dun_rot', 'fa_dun_semi',
       'p_aa_pp', 'hxl_tors', 'rama_prepro', 'gen_bonded', 'total','total_atom_depth', 'total_hb_to_lig']
        related_features = common_features
        all_df['total_hb_to_lig'] = 0
        for feature in list(all_df.columns):
            if 'hb_to' in feature:
                all_df['total_hb_to_lig'] += all_df[feature]
        all_df.to_csv(f'{design_dir}score_chosen_scaffolds_to_extract.csv') 
        make_dist_plots(all_df,related_features)

## 3_8. (optional) if okay number of designs got  at this step, can also select final design cretiera
- in general is good to check your designs to see if there is anything wrong. 

In [ ]:
# Check final designs by looking at designs at different percentile
quantiles = {
             'contact_molecular_surface':[False,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
              'dsasa':[False,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
              'ddg2':[True,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
              'holes_around_lig':[True,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]],
             'total_atom_depth':[True,[99.9,99,98,97,96,95,90,85,70,50,30,25,15,10,1]]
            } # select till quantiles are still represented based on ROC
for n,lig in enumerate(ligands):
    if lig == 'T44':
        design_dir= f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        os.chdir(design_dir)
        all_sc = f'{design_dir}score_chosen_scaffolds_to_extract.csv'
        all_df = pd.read_csv(all_sc)
        print(len(all_df))
        check_design_dir = f'{design_dir}check_designs_at_qantiles/'
        os.makedirs(check_design_dir,exist_ok=True)
        for feature in quantiles:
            all_df = all_df.sort_values(by=feature,ascending=quantiles[feature][0])
            for quantile in quantiles[feature][1]:
                if quantiles[feature][0]:
                    value = all_df.quantile(q=(100-quantile)/100,axis=0,interpolation='nearest')[feature]
                    print('___________________________________________')
                    print(f'! at quantile {feature} {quantile}: {value}, scaffold left: {len(all_df[all_df[feature]<=value])}')
                else:
                    value = all_df.quantile(q=quantile/100,axis=0,interpolation='nearest')[feature]
                    print('___________________________________________')
                    print(f'! at quantile {feature} {quantile}: {value}, scaffold left: {len(all_df[all_df[feature]>=value])}')
                pdb_row = all_df[all_df[feature]==value].head(1)
                pdb = pdb_row.pdb_path.values[0]
                if os.path.isfile(pdb):
                    os.chdir(check_design_dir)
                    cmd = f'cp {pdb} {feature}{quantile}.pdb'
                    subprocess.check_call(cmd,shell=True)

#### look at the selected designs and check what's the quantile to choose. Only keep the best ones, good enough for order

In [12]:
# Manually edit this
# feature:[quantile,ascending=T/F,actual_value] #should take top quantile
# These are Rosetta filters used for final selection
FINAL_CHOSE_QUANTILE = {
                  'T44':{'total_hb_to_lig':[5,False,5],
                         'contact_molecular_surface':[50,False,367],
                         'ddg2':[50,True,-26.6], #this one visually cannot distinguish
                         'dsasa':[70,False,0.976],
                         'holes_around_lig':[30,True,1.58],
                         'total_atom_depth':[50,True,6.16] #don't use this feature for CHD since checked before
                        }
                 }

In [ ]:
finalized_SEL = True
for n, lig in enumerate(ligands):
    if lig == 'T44':
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        all_df = pd.read_csv(f'{design_dir}score_chosen_scaffolds_to_extract.csv')
        chose_df = all_df.copy()
        print(f'{lig} start from {len(chose_df)}')
        for feature in FINAL_CHOSE_QUANTILE[lig]:
            chose_df = chose_df.sort_values(by=feature,ascending=FINAL_CHOSE_QUANTILE[lig][feature][1])
            value = FINAL_CHOSE_QUANTILE[lig][feature][2]
            if FINAL_CHOSE_QUANTILE[lig][feature][1]: #ascending=True
                chose_df = chose_df[chose_df[feature] <= value]
                print(f'{feature} at quantile {FINAL_CHOSE_QUANTILE[lig][feature][0]} less than value {value},',
                      f'left scaffolds {len(chose_df)}')
            else:
                chose_df = chose_df[chose_df[feature] >= value]
                print(f'{feature} at quantile {FINAL_CHOSE_QUANTILE[lig][feature][0]} greater than value {value},',
                      f'left scaffolds {len(chose_df)}')
        print(f'chose designs '+str(len(chose_df)/len(all_df)*100)+'%')
        print(f'chose scaffolds '+str(len(set(list(chose_df['cluster'])))))
        plot2pandas(chose_df,all_df,FINAL_CHOSE_QUANTILE[lig].keys(),'chose','all',ncols = 4)
        
        if finalized_SEL:
            # this is the file to include for final design selection as well
            chose_df.to_csv(f'{design_dir}final_chosen_scaffolds_to_resample_scaffolds.csv')
            #design these chosen docks for accurate MPNN resampling
            selected_dir = f'{design_dir}sel_for_af2/'
            os.makedirs(selected_dir,exist_ok=True)
            with_lig_dir = f'{selected_dir}with_lig/'
            os.makedirs(with_lig_dir,exist_ok=True)
            AF2_dir = f'{selected_dir}AF2_results/'
            os.makedirs(AF2_dir,exist_ok=True)
            chA_only = f'{selected_dir}chA_only/'
            os.makedirs(chA_only,exist_ok=True)
            for i,(j,row) in enumerate(chose_df.iterrows()):
                pdb_path = row['pdb_path']
                cmd = f'cp -n {pdb_path} {with_lig_dir}'
                subprocess.check_call(cmd,shell=True)
                cmd = f'grep " A " {pdb_path} > {chA_only}'+os.path.basename(pdb_path)
                subprocess.check_call(cmd,shell=True)
            
        #plot results
        plot2pandas(chose_df,all_df,FINAL_CHOSE_QUANTILE[lig].keys(),'chosen','all',ncols=3)

In [ ]:
## Check AF2 score for these designs
# ref: https://github.com/google-deepmind/alphafold.git
AF2 = '/software/conda/envs/mlfold/bin/python SM_af2/run_af2.py' # follow af2 instruction on docker/environment setting for this script
BATCH = 15 # request 32g, 200 aa need 2 g, can load 32/2 = 16 proteins at maximum
for n,lig in enumerate(ligands):
    if lig == 'T44':
        cmds = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        os.chdir(design_dir)
        chose_df = pd.read_csv(f'{design_dir}final_chosen_scaffolds_to_resample_scaffolds.csv')
        selected_dir = f'{design_dir}sel_for_af2/'
        with_lig_dir = f'{selected_dir}with_lig/'
        AF2_dir = f'{selected_dir}AF2_results/'
        chA_only = f'{selected_dir}chA_only/'
        for rotamer in list(set(chose_df.rotamer)):
            print(f'For {rotamer}:________________________________')
            rotamer_sel_df = chose_df[chose_df['rotamer']==rotamer]
            for cluster in list(set(rotamer_sel_df.cluster)):
                cluster_chose_df = rotamer_sel_df[rotamer_sel_df['cluster']==cluster]
                indi_cluster_dir = scaffold_df.loc[scaffold_df['cluster']==cluster,
                                                   'indi_cluster_dir'].values[0]
                if len(cluster_chose_df) > 0:
                    length = int(scaffold_df.loc[scaffold_df['cluster']==cluster,'nres'].values[0])
                    pdbs = list(cluster_chose_df['pdb_path'])
                    af2_indi_dir = f'{chA_only}{rotamer}/{indi_cluster_dir}'
                    af2_result_dir = f'{AF2_dir}{rotamer}/{indi_cluster_dir}'
                    os.makedirs(af2_indi_dir,exist_ok=True)
                    os.makedirs(af2_result_dir,exist_ok=True)
                    fastas = []
                    for i,pdb in enumerate(pdbs): 
                        fasta_f = f'{af2_indi_dir}'+os.path.basename(pdb)[:-4]+'.fasta'
                        af2_model = glob.glob(af2_result_dir+'/'+os.path.basename(pdb)[:-4]+\
                                              '*_model_*.pdb')
                        if len(af2_model) > 0:
                            subprocess.check_call(f'rm -f {fasta_f}',shell=True)
                        else:
                            record_gen = SeqIO.parse(pdb,"pdb-atom")
                            for record in record_gen:
                                seq = str(record.seq)
                            with open(fasta_f,'w') as f:
                                f.write('>'+os.path.basename(pdb)[:-4]+'\n'+seq)
                            fastas.append(fasta_f)
                    if len(fastas) > 0:
                        fasta_list = [fastas[i:i + BATCH] for i in range(0, len(fastas), BATCH)]
                        for fasta_fs in fasta_list:
                            batch_size = len(fasta_fs)
                            fasta_fs = ','.join(fasta_fs)
                            cmds.append(f'{AF2} --fasta_fs {fasta_fs} --out_dir {af2_result_dir} '+\
                                        f' --min_length {length} --max_length {length+5} --num_models 5 '+\
                                        f' --batch_size {batch_size} ')
        if len(cmds) > 0:
            cmd_f = f'{lig}_check_af2_cmd'
            with open(cmd_f,'w') as f:
                f.write('\n'.join(cmds))
            make_submit_file(cmds=cmd_f,submitfile=f'{cmd_f}.submit.sh',time="2-0:00:00",
            group_size=int(len(cmds)/4),logsfolder='logs',cpus=2,mem='32g',needs_gpu=True)
            SUBMIT = False
            if SUBMIT:
                subprocess.check_call(f'sbatch {cmd_f}.submit.sh',shell=True)

In [ ]:
## add AF2 plDDT scores and RMSD
for n,lig in enumerate(ligands):
    if lig == 'T44':
        cmds = []
        df_list = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        os.chdir(design_dir)
        chose_df = pd.read_csv(f'{design_dir}final_chosen_scaffolds_to_resample_scaffolds.csv')
        chose_df['rotamer'] = chose_df['pdb_path'].str.split('/').str[-4]
        for col in list(chose_df.columns):
            if 'model' in col:
                chose_df = chose_df.drop([col],axis=1)
        selected_dir = f'{design_dir}sel_for_af2/'
        with_lig_dir = f'{selected_dir}with_lig/'
        chA_dir = f'{selected_dir}chA_only/'
        AF2_dir = f'{selected_dir}AF2_results/'
        for rotamer in list(set(chose_df.rotamer)):
            print(f'For {rotamer}:________________________________')
            rotamer_sel_df = chose_df[chose_df['rotamer']==rotamer]
            for cluster in list(set(rotamer_sel_df.cluster)):
                cluster_chose_df = rotamer_sel_df[rotamer_sel_df['cluster']==cluster]
                indi_cluster_dir = scaffold_df.loc[scaffold_df['cluster']==cluster,
                                                   'indi_cluster_dir'].values[0]
                if len(cluster_chose_df) > 0:
                    length = int(scaffold_df.loc[scaffold_df['cluster']==cluster,'nres'].values[0])
                    pdbs = list(cluster_chose_df['pdb_path'])
                    af2_indi_dir = f'{chA_dir}{rotamer}/{indi_cluster_dir}'
                    af2_result_dir = f'{AF2_dir}{rotamer}/{indi_cluster_dir}'
                    for i,pdb in enumerate(pdbs): 
                        pdb_info = {}
                        af2_models = glob.glob(af2_result_dir+\
                                               os.path.basename(pdb)[:-4]+'*_model_*.pdb')
                        if len(af2_models) > 0:
                            chA = f'{af2_indi_dir}'+os.path.basename(pdb)
                            if not os.path.isfile(chA):
                                subprocess.check_call(f'grep " A " {pdb} > {chA}',shell=True)
                            for af2_model in af2_models:
                                with open(af2_model,'r') as f:
                                    for line in f:
                                        if 'plddt' in line:
                                            pdb_info[line.split()[0]] = [line.split()[-1]]
                                if 'model_4' in af2_model:
                                    pdb_info['unrelaxed_model_4'] = [af2_model]
                                    rmsd = get_RMSD(TMalign(chA,af2_model))
                                    pdb_info['rmsd_model4'] = rmsd
                            pdb_info['pdb_path'] = [pdb]
                            indi_df = pd.DataFrame.from_dict(pdb_info)
                            df_list.append(indi_df)
                        else:
                            continue
                    
        all_plddt_df = pd.concat(df_list)
        chose_df = chose_df.merge(all_plddt_df,how='left',on='pdb_path')
        chose_df.to_csv(f'{design_dir}final_chosen_scaffolds_to_resample_scaffolds.csv')
## Final filter

In [ ]:
## Copy good designs to another folder, these are for order
RMSD = 1.5 # 1-1.5 A is probably good
plDDT  = 85 # > 85 is good, only plddt is high enough, we can be more confident on sc.
cutoff_final = True
for n,lig in enumerate(ligands):
    if lig == 'T44':
        cmds = []
        df_list = []
        design_dir = f'{scratch_dir}3_prediction_ref2015_oldmethod/{lig}_prediction/'
        os.chdir(design_dir)
        chose_df = pd.read_csv(f'{design_dir}final_chosen_scaffolds_to_resample_scaffolds.csv')
        order_dir = f'{scratch_dir}9_for_order/{lig}_design/'
        enriched_design_dir = f'{order_dir}1st_round_enriched_from_good_clusters/'
        os.makedirs(enriched_design_dir,exist_ok=True)
        check = chose_df[~chose_df['unrelaxed_model_4'].isna()]
        sel_for_order = check[((check['model_1'] >= plDDT) | (check['model_2'] >=  plDDT) |\
                               (check['model_3'] >= plDDT) | (check['model_4'] >=  plDDT) |\
                               (check['model_5'] >=  plDDT)) & (check['rmsd_model4'] <= RMSD)]
        #plot results
        print(f'{len(sel_for_order)} designs also passed AF2, rate {len(sel_for_order)}/{len(chose_df)}={100*len(sel_for_order)/len(chose_df)}%')
        features = ['contact_molecular_surface','dsasa','holes_around_lig',\
                    'ddg2','total_hb_to_lig','total_atom_depth','model_4','rmsd_model4']
        plot2pandas(sel_for_order,chose_df,features,'pass_rosetta_af2','pass_rosetta',ncols=4)
        
        if cutoff_final:
            sel_for_order.to_csv(f'{order_dir}1st_orders_from_scaffold_enrichment.csv')
            for i,(j,row) in enumerate(sel_for_order.iterrows()):
                pdb = row['pdb_path']
                cmd = f'cp -n {pdb} {enriched_design_dir}'
                subprocess.check_call(cmd,shell=True)